In [2]:
# Importing dependencies.
import pandas as pd
import numpy as np
from citipy import citipy
from config import weather_api_key
import json
import requests

In [3]:
# Creating latitude and longitude sets
lats = np.random.uniform(low = -90, high = 90, size = 2000)
lngs = np.random.uniform(low = -180, high = 180, size = 2000)

lats_lngs = zip(lats, lngs)

coordinates = list(lats_lngs)

In [4]:
# City list holder
cities = []

# Finding nearest city and storing in holder
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If city is unique then add to list
    if city not in cities:
        cities.append(city)

# Checking amount of cities in list
len(cities)

737

In [5]:
# API call with OpenWeatherMap

# Base URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key 

# Empty weather data list
weather_data = []

# Loop through all the cities
for city in cities:
    
    # URL for each city
    weather_url = url + "&q=" + city
    
    # Running an API request for each of the cities
    try:
        
        # Parse the JSON and retrieve data
        target = requests.get(weather_url).json()
        
        # Parse out the data.
        country = target["sys"]["country"]
        lat = target["coord"]["lat"]
        lon = target["coord"]["lon"]
        temp = target["main"]["temp_max"]
        humidity = target["main"]["humidity"]
        cloud = target["clouds"]["all"]
        wind = target["wind"]["speed"]
        description = target["weather"][0]["description"]
    
        # Append the weather information into weather_data list
        weather_data.append({
            "City": city.title(),
            "Country": country,
            "Lat": lat,
            "Lng": lon,
            "Max Temp": temp,
            "Humidity": humidity,
            "Cloudiness": cloud,
            "Wind Speed": wind,
            "Current Description": description
        })
    # If an error is experienced, skip the city    
    except:
        pass

In [6]:
# Converting the array of dictionaries to a Pandas DataFrame
weather_df = pd.DataFrame(weather_data)
weather_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Shakawe,BW,-18.3667,21.8500,72.61,29,80,4.18,broken clouds
1,Qaanaaq,GL,77.4840,-69.3632,40.33,72,100,1.66,overcast clouds
2,Ushuaia,AR,-54.8000,-68.3000,40.66,45,0,14.97,clear sky
3,Barrow,US,71.2906,-156.7887,39.22,87,0,11.50,clear sky
4,Sur,OM,22.5667,59.5289,93.61,57,91,7.87,overcast clouds
5,Praya,CV,14.9215,-23.5087,79.34,61,75,17.27,broken clouds
6,Punta Arenas,CL,-53.1500,-70.9167,35.71,75,75,12.66,broken clouds
7,Butaritari,KI,3.0707,172.7902,82.72,76,76,20.45,light rain
8,Waddan,LY,29.1614,16.1390,102.38,11,0,18.88,clear sky
9,Sola,VU,-13.8833,167.5500,80.87,77,11,24.09,few clouds


In [8]:
# Create the output file and exporting in to CSV file
out_data = "WeatherPy_Database.csv"
weather_df.to_csv(out_data)